In [1]:
# noexport

!typech train_model.ipynb

[NbConvertApp] Converting notebook train_model.ipynb to python
train_model.py:330: error: Too few arguments for "iterateTrainingData"


In [30]:
from mkdata import *

In [3]:
import torch
import torch.nn as nn
import numpy as np
import os
import os.path

In [31]:
from model_selfattentionlstm import SelfAttentionLSTM

In [49]:
all_features_data = get_all_features_data()

In [33]:
#get_feature_names()

In [34]:
#from train_utils import sample_random_parameters
#parameters = sample_random_parameters([])
parameters = {'enabled_feature_list': get_feature_names()}

In [36]:
all_data_tensors = make_tensors_from_features(all_features_data, parameters)

In [37]:
all_features_data['8d2c9eb27dee2dc85bca705b'].keys()

dict_keys(['initial_difficulty', 'languages', 'difficulty_items'])

In [38]:
#training_data_all_features,dev_data_all_features,test_data_all_features = split_into_train_dev_test(all_features_data)
training_data_all_features,dev_data_all_features,test_data_all_features = split_into_train_dev_test(all_data_tensors)

In [39]:
def iterateTrainingData(training_data):
  output = []
  for data in training_data:
    category = tensor_to_difficulty(data['category'])
    #yield category,data['category'],data['feature']
    output.append((category,data['category'],data['feature']))
  np.random.shuffle(output)
  return output

criterion = nn.NLLLoss()


In [40]:
torch.cuda.device_count()

1

In [41]:
def save_model(rnn, criterion, epoch, loss, filename):
  torch.save({
    'epoch': epoch,
    'model_state_dict': rnn.state_dict(),
    'optimizer_state_dict': criterion.state_dict(),
    'loss': loss,
  }, filename)

#print(criterion.state_dict())

In [42]:
def tensor_to_difficulty(tensor):
  difficulty_idx = tensor[0].data.cpu().numpy()
  return ['nothing', 'easy', 'medium', 'hard'][difficulty_idx]

def categoryFromOutput(output):
  top_n,top_i = output.topk(1)
  category_i = top_i[0].item()
  return ['nothing','easy','medium','hard'][category_i],category_i

#print(output.topk(1))
#print(categoryFromOutput(output))

print(make_tensor_from_chosen_difficulty('hard'))
print(tensor_to_difficulty((make_tensor_from_chosen_difficulty('hard'))))

tensor([3])
hard


In [43]:
def trainTransformer(category_tensor, line_tensor):
    
    model.zero_grad()
    
    # TODO: construct mask and lengths for batch size == 1
    lengths = torch.tensor([line_tensor.size()[0]])
    mask = torch.zeros(line_tensor.size()[0], line_tensor.size()[1], 1, dtype=torch.float)
    device = (torch.device('cuda') if torch.cuda.is_available() else
                   torch.device('cpu'))
    mask = mask.to(device)
    line_tensor = line_tensor.to(device)
    category_tensor = category_tensor.to(device)
    # END TODO
#     print(line_tensor.size())
#     print(category_tensor.size())
#     print(lengths.size())
#     print(mask.size())
    output = model(line_tensor, lengths, mask)
    # print(category_tensor.size())
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in model.parameters():
        if p.grad is None:
          continue
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [44]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

In [45]:
feature_names = get_feature_names()
num_features = get_num_features(feature_names)
num_prior_entries = 10
enable_current_difficulty = False

In [48]:
# model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
# all_training_items = iterateTrainingData(training_data_all_features)
# print('sample_difficulty_every_n_visits', sample_difficulty_every_n_visits)
# current_loss = 0
# for idx,training_item in enumerate(all_training_items):
#   (category,category_tensor, line_tensor) = training_item
#   if line_tensor.size()[0] == 0:
#     continue
#   output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
#   current_loss += loss
#   if idx % 1000 == 0:
#     #pass
#     print(epoch, ':', idx, '/', len(all_training_items))
#     # Print iter number, loss, name and guess
#   if epoch % print_every == 0:
#     guess, guess_i = categoryFromOutput(output)
#     correct = '✓' if guess == category else '✗ (%s)' % category
#     #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))


sample_difficulty_every_n_visits 1
1 : 0 / 270173
1 : 1000 / 270173


KeyboardInterrupt: 

In [ ]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []

for sample_difficulty_every_n_visits in range(1, 1001):
  epoch = 1
  outfile = 'model_attention_nohistory_fulldata_nhidden512_sample_difficulty_every_n_visits_' + str(sample_difficulty_every_n_visits) + '_v10_epoch' + str(epoch) + '.pt'
  if os.path.exists(outfile):
    continue
  print('making training data for sample_difficulty_every_n_visits', sample_difficulty_every_n_visits)
  all_features_data = get_all_features_data()
  all_data_tensors = make_tensors_from_features(all_features_data, {
    'enabled_feature_list': feature_names,
    'num_prior_entries': num_prior_entries,
    'enable_current_difficulty': enable_current_difficulty,
    'sample_difficulty_every_n_visits': sample_difficulty_every_n_visits,
  })
  training_data_all_features,dev_data_all_features,test_data_all_features = split_into_train_dev_test(all_data_tensors)
  #training_data = make_tensors_from_features(training_data_all_features, {
  #  'enabled_feature_list': feature_names,
  #  'num_prior_entries': num_prior_entries,
  #  'enable_current_difficulty': enable_current_difficulty,
  #  'sample_difficulty_every_n_visits': sample_difficulty_every_n_visits,
  #})
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  all_training_items = iterateTrainingData(training_data_all_features)
  print('sample_difficulty_every_n_visits', sample_difficulty_every_n_visits)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, outfile)



making training data for sample_difficulty_every_n_visits 1


In [52]:
#print((all_features_data).keys())

dict_keys(['8d2c9eb27dee2dc85bca705b', '4a5f492c987b873871be4c12', '78cc8c70b8be7412e00a1575', '8642884bd204eafc660a04a2', '9ca53f1f33ac3e25c2e3f3d2', '6e055258e3625ab1db57f0fa', '9c00827409f8c296d67f78b4', '57ddb2820db2de361757d21b', 'af8454409cde73bead9d1071', 'eccfe39b9019a3f566346399', 'f32cf25130f6fa751892960e', '8f77439cf19de38abfd4a417', '3bc557eda64de5ff9d2e6fca', '16043b6d098957d10448c009', 'edc3437cabf565d7a8c2f96c', '88550c3144f8e1a6e84e83e2', '3bc201cd34fc67185861245e', '472a909dcf35ed6f1bc6da82', '150f93e5b7dea556c943813e', '93b71831840fb44e554b6e36', 'cb266f371364b060f117b6a1', '4e69f3aebac131196a8f2912', '834ab49ae4351c22a1d49c2d', '43a6fdadccd61a51caf5b588', 'f6e9666fa6fd68a8ff0e2781', '48df425ff6b2910b961070e5', 'ffdc2b2d686dbf9c8c621b35', '9595707af25dc9cf08d162f2', '414492408538c282663be78d', '8880741c0eb16b95ba062848', 'cc43e1c331c9d74bf7b00fc6', 'ae97c36156bb44a178a6519d', '2065bc9f616de4053a71c97b', '1ac79e4a7430d335fd2995e7', '99ae501cd07551f9d6628470', '477fc312

In [53]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []

for sample_difficulty_every_n_visits in range(1, 1001):
  epoch = 1
  outfile = 'model_attention_nohistory_fractiondata10_nhidden512_sample_every_n_visits_' + str(sample_difficulty_every_n_visits) + '_v10_epoch' + str(epoch) + '.pt'
  if os.path.exists(outfile):
    continue
  print('making training data for sample_difficulty_every_n_visits', sample_difficulty_every_n_visits)
  all_features_data = get_all_features_data()
  all_data_tensors = make_tensors_from_features(all_features_data, {
    'enabled_feature_list': feature_names,
    'num_prior_entries': num_prior_entries,
    'enable_current_difficulty': enable_current_difficulty,
    'sample_difficulty_every_n_visits': sample_difficulty_every_n_visits,
  })
  training_data_all_features,dev_data_all_features,test_data_all_features = split_into_train_dev_test(all_data_tensors)
  training_data_all_features = training_data_all_features[:int(len(training_data_all_features) / 10)]
  #training_data = make_tensors_from_features(training_data_all_features, {
  #  'enabled_feature_list': feature_names,
  #  'num_prior_entries': num_prior_entries,
  #  'enable_current_difficulty': enable_current_difficulty,
  #  'sample_difficulty_every_n_visits': sample_difficulty_every_n_visits,
  #})
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  all_training_items = iterateTrainingData(training_data_all_features)
  print('sample_difficulty_every_n_visits', sample_difficulty_every_n_visits)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, outfile)



making training data for sample_difficulty_every_n_visits 1
sample_difficulty_every_n_visits 1
1 : 0 / 27017
1 : 1000 / 27017
1 : 2000 / 27017
1 : 3000 / 27017
1 : 4000 / 27017
1 : 5000 / 27017
1 : 6000 / 27017
1 : 7000 / 27017
1 : 8000 / 27017
1 : 9000 / 27017
1 : 10000 / 27017
1 : 11000 / 27017
1 : 12000 / 27017
1 : 13000 / 27017
1 : 14000 / 27017
1 : 15000 / 27017
1 : 16000 / 27017
1 : 17000 / 27017
1 : 18000 / 27017
1 : 19000 / 27017
1 : 20000 / 27017
1 : 21000 / 27017
1 : 22000 / 27017
1 : 23000 / 27017
1 : 24000 / 27017
1 : 25000 / 27017
1 : 26000 / 27017
1 : 27000 / 27017
making training data for sample_difficulty_every_n_visits 2


KeyboardInterrupt: 

In [ ]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []

for sample_every_n_visits in range(1, 11):
  print('making training data for sample_every_n_visits', sample_every_n_visits)
  training_data = make_tensors_from_features(training_data_all_features[:int(len(training_data_all_features) / 10)], {
    'enabled_feature_list': feature_names,
    'num_prior_entries': num_prior_entries,
    'enable_current_difficulty': enable_current_difficulty,
    'sample_every_n_visits': sample_every_n_visits,
  })
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  epoch = 1
  all_training_items = iterateTrainingData(training_data)
  print('sample_every_n_visits', sample_every_n_visits)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, 'model_attention_nohistory_fractiondata10_nhidden512_sample_every_n_visits_' + str(sample_every_n_visits) + '_v10_epoch' + str(epoch) + '.pt')



In [ ]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []
disable_difficulty_history = True
sample_every_n_visits = 1

#for sample_every_n_visits in range(1, 11):
if True:
  print('making training data for disable_difficulty_history', disable_difficulty_history)
  training_data = make_tensors_from_features(training_data_all_features[:int(len(training_data_all_features) / 10)], {
    'enabled_feature_list': feature_names,
    'num_prior_entries': num_prior_entries,
    'enable_current_difficulty': enable_current_difficulty,
    'sample_every_n_visits': sample_every_n_visits,
    'disable_difficulty_history': disable_difficulty_history,
  })
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  epoch = 1
  all_training_items = iterateTrainingData(training_data)
  print('sample_every_n_visits', sample_every_n_visits)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, 'model_attention_nohistory_fractiondata10_nhidden512_disable_difficulty_history_v10_epoch' + str(epoch) + '.pt')



In [ ]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []
disable_prior_visit_history = True

#for sample_every_n_visits in range(1, 11):
if True:
  print('making training data for disable_prior_visit_history', disable_prior_visit_history)
  training_data = make_tensors_from_features(training_data_all_features[:int(len(training_data_all_features) / 10)], {
    'enabled_feature_list': feature_names,
    'num_prior_entries': num_prior_entries,
    'enable_current_difficulty': enable_current_difficulty,
    'sample_every_n_visits': sample_every_n_visits,
    'disable_prior_visit_history': disable_prior_visit_history,
  })
  model = SelfAttentionLSTM({'word_embed_size': num_features, 'window_embed_size': 128})
  epoch = 1
  all_training_items = iterateTrainingData(training_data)
  print('sample_every_n_visits', sample_every_n_visits)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, 'model_attention_nohistory_fractiondata10_nhidden512_disable_prior_visit_history_v10_epoch' + str(epoch) + '.pt')



In [ ]:
n_iters = 100
print_every = 1
plot_every = 1
all_losses = []

model.train()
for epoch in range(1, n_iters + 1):
  all_training_items = iterateTrainingData()
  print('iteration', epoch)
  current_loss = 0
  for idx,training_item in enumerate(all_training_items):
    (category,category_tensor, line_tensor) = training_item
    #print(line_tensor.size())
    if line_tensor.size()[0] == 0:
      continue
    output, loss = trainTransformer(category_tensor, line_tensor.permute(1,0,2))
    current_loss += loss
    if idx % 1000 == 0:
      #pass
      print(epoch, ':', idx, '/', len(all_training_items))
      # Print iter number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to list of losses
    #if iter % plot_every == 0:
    #    all_losses.append(current_loss / plot_every)
    #    #current_loss = 0
  #rnn.zero_grad()
  all_losses.append(current_loss / plot_every)
  save_model(model, criterion, epoch, current_loss, 'model_attention_nohistory_nhidden512_v10_epoch' + str(epoch) + '.pt')